# Faculty Survey

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

In [2]:
df = pd.read_csv("faculty_survey_anonymized.csv")

alias_map = {f"col{i}": col for i, col in enumerate(df.columns, start=1)}
reverse_map = {v: k for k, v in alias_map.items()}

df_alias = df.rename(columns=reverse_map)

print("Alias to Original Header Mapping:")
for alias, original in alias_map.items():
    print(f"{alias}: {original}")

df_alias = df_alias.fillna("No Response")

Alias to Original Header Mapping:
col1: Start Date
col2: End Date
col3: Progress
col4: Duration (in seconds)
col5: Finished
col6: Recorded Date
col7: What is your gender?
col8: What is your race/ethnicity?
col9: How many years have you been teaching for Augustin College?
col10: What Augustin College programs do you teach in?
col11: Which undergraduate programs do you currently teach in?
col12: Which graduate programs do you currently teach in?
col13: How important are the following aspects for promoting Augustin College to future students?* - Flexible learning Options (e.g., online, evening, hybrid)
col14: How important are the following aspects for promoting Augustin College to future students?* - Career-focused programs aligned with workforce needs
col15: How important are the following aspects for promoting Augustin College to future students?* - Competitive tuition and robust financial aid support
col16: How important are the following aspects for promoting Augustin College to futu

## Demographics

In [3]:
fig1 = px.pie(df_alias, names = "col7", title=alias_map["col7"], color_discrete_sequence=["#8A100B", "#BC9B6A"]
)

fig1.show()

In [4]:
fig2 = px.pie(df_alias, names = "col8", title=alias_map["col8"],color_discrete_sequence=["#8A100B", "#BC9B6A"]
)

fig2.show()

In [5]:
fig3 = px.bar(
    df_alias,
    x="col9",
    title=alias_map["col9"], 
    color_discrete_sequence=["#8A100B", "#BC9B6A"]  # show original column name as chart title
)

fig3.update_layout(
    xaxis={'categoryorder': 'category ascending'}
)

fig3.show()

In [6]:
fig4 = px.pie(
    df_alias,
    names="col10",
    hole=0.4,
    title=alias_map["col10"],
    color_discrete_sequence=["#8A100B", "#BC9B6A"]
)

fig4.show()

## Survey Results

In [7]:
importance_map = {
    "extremely important": 1,
    "very important": 2,
    "moderately important": 3,
    "slightly important": 4,
    "not at all important": 5,
    "no response": np.nan   # treat "No Response" as missing
}

In [8]:
df_mapped = df_alias.copy()

part1 = [f"col{i}" for i in range(13, 62)]
part2 = [f"col{i}" for i in range(63, 95)]
part3 = [f"col{i}" for i in range(96, 105)]

text_cols = part1 + part2 + part3

df_mapped[text_cols] = df_mapped[text_cols].map(
    lambda x: x.strip().lower() if isinstance(x, str) else x
)

df_mapped[text_cols] = df_mapped[text_cols].replace(importance_map)

df_mapped[text_cols] = df_mapped[text_cols].apply(pd.to_numeric, errors="coerce")

/var/folders/bj/bctg9sf9731d8nt80_gfnqxm0000gn/T/ipykernel_40689/3599180202.py:13: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [9]:
# Column-wise sums (ignores NaN)
col_sums = df_mapped[text_cols].sum()

# Dictionary of column → sum
col_sum_dict = col_sums.to_dict()

In [10]:
sorted_totals = dict(sorted(col_sum_dict.items(), key=lambda x: x[1]))

for col, total in sorted_totals.items():
    print(f"{col}: {total}")


col71: 146.0
col87: 151.0
col100: 151.0
col58: 153.0
col57: 155.0
col28: 161.0
col61: 165.0
col38: 167.0
col13: 168.0
col93: 168.0
col104: 168.0
col94: 169.0
col98: 169.0
col89: 170.0
col75: 171.0
col19: 172.0
col86: 172.0
col14: 174.0
col20: 174.0
col52: 174.0
col97: 175.0
col46: 176.0
col54: 176.0
col15: 177.0
col88: 177.0
col77: 180.0
col76: 181.0
col59: 182.0
col101: 182.0
col43: 184.0
col34: 185.0
col60: 186.0
col92: 187.0
col69: 190.0
col91: 190.0
col68: 195.0
col17: 197.0
col90: 197.0
col103: 197.0
col99: 198.0
col32: 199.0
col45: 199.0
col84: 199.0
col102: 199.0
col39: 201.0
col41: 205.0
col67: 205.0
col72: 205.0
col56: 207.0
col79: 208.0
col55: 215.0
col74: 216.0
col33: 219.0
col80: 219.0
col96: 219.0
col21: 222.0
col85: 222.0
col63: 223.0
col50: 225.0
col16: 226.0
col35: 233.0
col82: 233.0
col37: 235.0
col83: 235.0
col47: 236.0
col81: 242.0
col18: 244.0
col29: 244.0
col44: 250.0
col40: 253.0
col64: 255.0
col36: 257.0
col53: 257.0
col48: 264.0
col30: 265.0
col65: 275.0
col78: 

In [11]:
topic_map = {}
for alias, original in alias_map.items():
    if " - " in original:
        topic_map[alias] = original.split(" - ", 1)[1]  # take part after dash
    else:
        topic_map[alias] = original  # fallback: just use full text




In [12]:
sorted_totals = dict(sorted(col_sum_dict.items(), key=lambda x: x[1]))

for col, total in sorted_totals.items():
    topic = topic_map.get(col, "")
    print(f"{col} | {topic} | {total}")

col71 | Career Center | 146.0
col87 | Opportunities for career networking | 151.0
col100 | Focusing on real-world impact and professional advancement | 151.0
col58 | Strong support systems | 153.0
col57 | High academic standards | 155.0
col28 | Veritus Imperial University's overall academic reputation | 161.0
col61 | Clear and timely feedback | 165.0
col38 | Online asynchronous learning opportunities (flexible, self-paced online classes) | 167.0
col13 | Flexible learning Options (e.g., online, evening, hybrid) | 168.0
col93 | Networking | 168.0
col104 | Aligning with the evolving challenges and opportunities of the future workforce | 168.0
col94 | Clearer alignment between academic programs and professional goals | 169.0
col98 | Preparing students to be ethical leaders in a complex world | 169.0
col89 | Career advising | 170.0
col75 | Financial Aid | 171.0
col19 | Strong Veritus Imperial University reputation and alumni network | 172.0
col86 | Expanded personalized career guidance tail

In [13]:
# Build DataFrame from sorted totals
results_faculty = pd.DataFrame([
    {
        "alias": col,
        "topic": topic_map.get(col, ""),
        "total_score": total
    }
    for col, total in sorted(col_sum_dict.items(), key=lambda x: x[1])
])

# Save to CSV
results_faculty.to_csv("results_faculty.csv", index=False)

print("✅ Exported results_faculty.csv")

✅ Exported results_faculty.csv
